In [306]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
import numpy as np
import pandas as pd
import mappings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, QuantileTransformer # Or MinMaxScaler
from keras.preprocessing.sequence import pad_sequences

In [307]:
year = 2025
df = pd.read_csv(f"final_data_processed_{year}.csv")
df.head(3)

,LapNumber,Driver,LapTime,Compound,TyreLife,Stint,FreshTyre,Team,TrackStatus,Position,...,Location,Year,Track,Age,Exprience,Achievements,AchievementsByTime,LastYearDriverPoints,LastYearTeamPoints,IsDriverActive
0,1,1,119.392,4,1.0,1.0,1,1.0,13,5.0,...,Melbourne,2025,1,28.0,211.0,3059.0,14.49763,437,589,1
1,3,1,0.000,4,3.0,2.0,0,1.0,9,2.0,...,Melbourne,2025,1,28.0,211.0,3059.0,14.49763,437,589,1
2,4,1,0.000,4,4.0,3.0,0,1.0,9,2.0,...,Melbourne,2025,1,28.0,211.0,3059.0,14.49763,437,589,1


In [308]:
df.isna().sum()

LapNumber               0
Driver                  0
LapTime                 0
Compound                0
TyreLife                0
Stint                   0
FreshTyre               0
Team                    0
TrackStatus             0
Position                0
Sector1Time             0
Sector2Time             0
Sector3Time             0
SpeedI1                 0
SpeedI2                 0
SpeedFL                 0
SpeedST                 0
SessionTime             0
AirTemp                 0
Humidity                0
Pressure                0
Rainfall                0
TrackTemp               0
WindDirection           0
WindSpeed               0
RpmAvg                  0
RpmMin                  0
RpmMax                  0
SpeedAvg                0
SpeedMedian             0
SpeedMin                0
SpeedMax                0
ThrottleAvg             0
ThrottleMin             0
ThrottleMax             0
nGearAvg                0
nGearMin                0
nGearMax                0
BrakeCount  

Sometimes there would be some NaN values which needs to be handled. In this case we only have these NaN values on Team column.
We can handle this easily by checking TeamId column.

In [309]:
df.loc[df['Team'].isna()]['TeamId'].unique()

array([], dtype=object)

So these NaN values are rb (racing bulls) - The id for 'rb' team is 5

In [456]:
df = pd.read_csv(f"final_data_2025.csv")
df_new = df.copy()
# convert two int columns to float
df_new[['BrakeCount', 'DrsCount']] = df_new[['BrakeCount', 'DrsCount']].astype(float)
# convert pit in time and pit out time to int
# df_new[['PitInTime', 'PitOutTime']] = df_new['PitInTime', 'PitOutTime'].astype(int)
# feature engineering
years=df_new['Year'].unique()
for year in years:
    print(f"Year processing is: {year}")
    year_df = df_new[df_new['Year'] == year].copy()
    # get Locations
    locations = year_df['Location'].unique()
    for i, loc in enumerate(locations):
        location_df = year_df[year_df['Location'] == loc].copy()
        if i % 10 == 0:
            print(f"Location processing is: {i} / {len(locations)}")

        # filter the location_df to the rows that doesn't have pit in and pit out times
        # location_dff = location_df[(location_df['PitInTime'] == 0) & (location_df['PitOutTime'] == 0)]
        # get the average of car data for the lap for all drivers
        lap_avg = location_df[['RpmAvg', 'RpmMin', 'RpmMax', 'SpeedAvg', 'SpeedMedian', 'SpeedMin', 'SpeedMax',
                            'ThrottleAvg','ThrottleMin', 'ThrottleMax', 'nGearAvg', 'nGearMin', 'nGearMax',
                            'BrakeCount', 'DrsCount','Sector1Time', 'Sector2Time', 'Sector3Time', 'SpeedI1', 
                            'SpeedI2', 'SpeedFL','SpeedST', 'SessionTime', 'LapTime']].mean()
        # compare all drivers to the average of the lap and add modify the columns in place
        for col in lap_avg.index:
            location_df[col] = location_df[col] - lap_avg[col]
            # add the lap_df to the df_new dataframe
            df_new.loc[(df_new['Year']==year)&(df_new['Location']==loc), col] = location_df[col]
        
df_new['IsDriverActive'] = 0
df_new.to_csv('final_data_processed_2025.csv', index=False)


Year processing is: 2025
Location processing is: 0 / 3


In [577]:
df.loc[df['Team'].isna(), 'Team'] = 5
# df.loc[df['LastYearTeamPoints'].isna(), 'LastYearTeamPoints'] = 46
df.isna().sum()

LapNumber             0
Driver                0
LapTime               0
Compound              0
TyreLife              0
Stint                 0
PitInTime             0
PitOutTime            0
FreshTyre             0
Team                  0
TrackStatus           0
Position              0
Sector1Time           0
Sector2Time           0
Sector3Time           0
SpeedI1               0
SpeedI2               0
SpeedFL               0
SpeedST               0
SessionTime           0
AirTemp               0
Humidity              0
Pressure              0
Rainfall              0
TrackTemp             0
WindDirection         0
WindSpeed             0
RpmAvg                0
RpmMin                0
RpmMax                0
SpeedAvg              0
SpeedMedian           0
SpeedMin              0
SpeedMax              0
ThrottleAvg           0
ThrottleMin           0
ThrottleMax           0
nGearAvg              0
nGearMin              0
nGearMax              0
BrakeCount            0
DrsCount        

In [311]:
df['IsDriverActive'] = 0
df.loc[df['LastYearDriverPoints'] > 20, 'IsDriverActive'] = 1

Loading Year 2025 data and select a track and first 20 laps

In [353]:
test_part_df = df.loc[(df['Year'] == 2025)&(df['Driver']==31)].copy()

test_part_df = test_part_df.loc[(test_part_df['Location'] == 'Suzuka') & (test_part_df['LapNumber'] < 40), :].copy()
len(test_part_df)

39

Loading trained objects

In [475]:
import joblib

scaler = joblib.load('scaler.pkl')

In [314]:
# # --- Hyperparameters ---
# NUM_DRIVERS = 45

# # --- columns ---
# numerical_feature_cols = ['LapTime', 'TyreLife', 'Stint','FreshTyre', 'Position', 'Sector1Time','Sector2Time', 'Sector3Time', 'SpeedI1', 'SpeedI2', 'SpeedFL',
#     'SpeedST', 'SessionTime', 'AirTemp', 'Humidity', 'Pressure', 'Rainfall','TrackTemp', 'WindDirection', 'WindSpeed', 'RpmAvg', 'RpmMin', 'RpmMax',
#     'SpeedAvg', 'SpeedMedian', 'SpeedMin', 'SpeedMax', 'ThrottleAvg','ThrottleMin', 'ThrottleMax', 'nGearAvg', 'nGearMin', 'nGearMax',
#     'BrakeCount', 'DrsCount', 'nGearMode',  'GridPosition', 'Age', 'Exprience','Achievements', 'AchievementsByTime', 'LastYearDriverPoints', 'LastYearTeamPoints']
# driver_id_col = 'Driver'
# driver_active_mask_col = 'IsDriverActive'
# team_id_col = 'Team'
# compound_id_col = 'Compound'
# track_status_col = 'TrackStatus'
# track_id_col = 'Track'


# # --- Define Race Identifier ---
# # Use a combination of columns that uniquely identifies each race event
# race_id_cols = ['Year', 'Location'] 
# # --- Define Target Column ---
# target_col = 'FinalPosition'

# # --- Function to Generate Sequences and Targets ---
# def generate_sequences(df, numerical_cols, id_cols, active_flag_col, target_col, race_group_cols, max_drivers):
#     """
#     Generates sequences lap-by-lap for each driver in each race.

#     Args:
#         df (pd.DataFrame): DataFrame containing lap data for multiple races.
#         numerical_cols (list): List of numerical feature column names.
#         id_cols (list): List of categorical ID column names (already encoded).
#         target_col (str): Name of the column containing the final driver position.
#         race_group_cols (list): Columns to group by to identify individual races.
#         max_drivers (int): The fixed number of drivers for the output vector.

#     Returns:
#         tuple: (list_of_numerical_sequences, list_of_id_sequences_lists, list_of_target_vectors)
#                - list_of_id_sequences_lists contains one list per ID type.
#     """
#     all_numerical_sequences = []
#     # Create a list of lists, one for each ID type
#     all_id_sequences = [[] for _ in id_cols]
#     all_targets = []
#     all_active_flags = []

#     # Group by each unique race
#     grouped_races = df.groupby(race_group_cols)
#     num_races = len(grouped_races)
#     print(f"Processing {num_races} races...")

#     for i, (race_info, race_df) in enumerate(grouped_races):
#         if (i + 1) % 10 == 0: # Print progress
#              print(f"  Processing race {i+1}/{num_races}...")

#         # Sort race by lap number just in case
#         race_df = race_df.sort_values('LapNumber')

#         # --- Determine the target vector for this race ---
#         # Get the final position for each driver in THIS race
#         # Group by driver, get the LAST recorded final position (should be consistent)
#         final_positions = race_df.groupby(driver_id_col)[target_col].last()

#         # Create a target score vector (e.g., higher score for better position)
#         target_scores = {}
#         for driver_id, position in final_positions.items():
#             # Simple scoring: Higher score is better. 
#             # I have put non finishers as 20
#             if position == 20:
#                 score = 0
#             else:
#                 score = 20 - position + 1
#             target_scores[int(driver_id)] = score # Ensure driver_id is int key

#         # Create the final target vector of size max_drivers
#         # Initialize with 0 (score for drivers not present or DNF)
#         race_target_vector = np.zeros(max_drivers)

#         for driver_id_encoded, score in target_scores.items():
#             # Place the score at the index matching the driver's global ID
#             if driver_id_encoded < max_drivers: # Safety check
#                 race_target_vector[driver_id_encoded] = score
#             # No 'else' needed, IDs beyond the known range are ignored

#         # --- Generate sequences for each driver in this race ---
#         drivers_in_race = race_df[driver_id_col].unique()
#         for driver_id in drivers_in_race:
#             driver_laps_df = race_df[race_df[driver_id_col] == driver_id].sort_values('LapNumber')
#             max_lap = driver_laps_df['LapNumber'].max()

#             # Generate sequence for each lap completed by the driver
#             for k in range(1, int(max_lap) + 1):
#                 # Get data for laps 1 to k for this driver
#                 current_sequence_df = driver_laps_df[driver_laps_df['LapNumber'] <= k]

#                 # Extract numerical features for the sequence
#                 numerical_sequence = current_sequence_df[numerical_cols].values
                
#                 all_numerical_sequences.append(numerical_sequence)

#                 # Extract each ID feature for the sequence
#                 for idx, id_col in enumerate(id_cols):
#                     id_sequence = current_sequence_df[id_col].values
                    
#                     all_id_sequences[idx].append(id_sequence)

#                 # Extract Active Flag for drivers
#                 active_flag_sequence = current_sequence_df[active_flag_col].values.astype(np.float32)
#                 all_active_flags.append(active_flag_sequence)

#                 # The target is the same for all sequences from this race
#                 all_targets.append(race_target_vector)

#     print(f"Finished generating {len(all_targets)} sequences.")
#     # Return numerical sequences, a list containing lists of sequences for each ID type, and targets
#     return all_numerical_sequences, all_id_sequences, all_active_flags, all_targets

# def prediction(pred, y_test):
#     drivers_mapping = mappings.drivers.copy()
#     driver_scores_prediction = {}
#     driver_scores_actual = {}

#     for name , id in drivers_mapping.items():
#         driver_scores_prediction[name] = round(list(pred)[0][id], 2)
#         driver_scores_actual[name] = int(list(y_test)[0][id])

#     print(driver_scores_prediction)
#     print(driver_scores_actual)
#     return driver_scores_prediction, driver_scores_actual

# def final_forecast_ranking(driver_scores_prediction, driver_scores_actual):
#     df_pred = pd.DataFrame(sorted(driver_scores_prediction.items(), key=lambda item: item[1], reverse=True), columns=['Driver', 'Score'])
#     df_actual = pd.DataFrame(sorted(driver_scores_actual.items(), key=lambda item: item[1], reverse=True), columns=['Driver', 'Score'])
#     dff = pd.concat([df_pred, df_actual], axis=1)
#     dff.columns= ['Driver_Pred', 'Score_Pred', 'Driver_Actual', 'Score_Actual']
#     return dff

# # --- Evaluate on Test Set (Optional but Recommended) ---
# # 1. Generate test sequences and pad them using the SAME scaler and MAX_SEQUENCE_LENGTH
# MAX_SEQUENCE_LENGTH = 78
# test_df = test_part_df.copy()
# test_df[numerical_feature_cols] = scaler.transform(test_df[numerical_feature_cols])

# print("\nPreparing Test Data...")
# X_test_num_seq, X_test_id_seq_lists, X_train_active_flags, y_test_list = generate_sequences(
#     test_df, numerical_feature_cols, [driver_id_col, team_id_col, compound_id_col, track_status_col, track_id_col],
#     driver_active_mask_col,
#     target_col, race_id_cols, NUM_DRIVERS
# )

# X_test_num_padded = pad_sequences(X_test_num_seq, maxlen=MAX_SEQUENCE_LENGTH, padding='pre', dtype='float32', value=0.0)
# X_test_ids_padded = [
#     pad_sequences(id_list, maxlen=MAX_SEQUENCE_LENGTH, padding='pre', value=0)
#     for id_list in X_test_id_seq_lists
# ]

# X_test_active_padded = pad_sequences(X_train_active_flags, maxlen=MAX_SEQUENCE_LENGTH, padding='pre', dtype='float32', value=0.0)

# y_test = np.array(y_test_list)
# X_test_inputs = [X_test_num_padded] + X_test_ids_padded + [X_test_active_padded]
# # X_test_inputs_dict = { ... } # Dictionary version

# # 2. Load the best saved model
# best_model = keras.models.load_model("best_f1_model1.keras") # error here because of the name

# # 3. Evaluate
# test_loss, test_mae = best_model.evaluate(X_test_inputs, y_test) # error here because of the name
# print(f"\nTest Loss: {test_loss:.4f}")
# print(f"Test MAE: {test_mae:.4f}")

# pred = best_model.predict(X_test_inputs)
# pred, actual = prediction(pred, y_test)
# dff = final_forecast_ranking(pred, actual)
# dff

In [315]:
# # --- Hyperparameters ---
# NUM_DRIVERS = 45

# # --- columns ---
# numerical_feature_cols = ['LapTime', 'TyreLife', 'Stint','FreshTyre',  'Sector1Time','Sector2Time', 'Sector3Time', 'SpeedI1', 'SpeedI2', 'SpeedFL',
#     'SpeedST', 'SessionTime', 'AirTemp', 'Humidity', 'Pressure', 'Rainfall','TrackTemp', 'WindDirection', 'WindSpeed', 'RpmAvg', 'RpmMin', 'RpmMax',
#     'SpeedAvg', 'SpeedMedian', 'SpeedMin', 'SpeedMax', 'ThrottleAvg','ThrottleMin', 'ThrottleMax', 'nGearAvg', 'nGearMin', 'nGearMax',
#     'BrakeCount', 'DrsCount', 'nGearMode',   'Age', 'Exprience','Achievements', 'AchievementsByTime',
#     'LastYearDriverPoints', 'LastYearTeamPoints',
#     # 'Position', 'GridPosition',
#     ]
# driver_id_col = 'Driver'
# driver_active_mask_col = 'IsDriverActive'
# team_id_col = 'Team'
# compound_id_col = 'Compound'
# track_status_col = 'TrackStatus'
# track_id_col = 'Track'
# position_col = 'Position'
# grid_position_col = 'GridPosition'

# # --- Define Race Identifier ---
# # Use a combination of columns that uniquely identifies each race event
# race_id_cols = ['Year', 'Location'] 
# # --- Define Target Column ---
# target_col = 'FinalPosition'

# # --- Function to Generate Sequences and Targets ---
# def generate_sequences(df, numerical_cols, id_cols, active_flag_col, target_col, race_group_cols, max_drivers):
#     """
#     Generates sequences lap-by-lap for each driver in each race.

#     Args:
#         df (pd.DataFrame): DataFrame containing lap data for multiple races.
#         numerical_cols (list): List of numerical feature column names.
#         id_cols (list): List of categorical ID column names (already encoded).
#         target_col (str): Name of the column containing the final driver position.
#         race_group_cols (list): Columns to group by to identify individual races.
#         max_drivers (int): The fixed number of drivers for the output vector.

#     Returns:
#         tuple: (list_of_numerical_sequences, list_of_id_sequences_lists, list_of_target_vectors)
#                - list_of_id_sequences_lists contains one list per ID type.
#     """
#     all_numerical_sequences = []
#     # Create a list of lists, one for each ID type
#     all_id_sequences = [[] for _ in id_cols]
#     all_targets = []
#     all_active_flags = []

#     # Group by each unique race
#     grouped_races = df.groupby(race_group_cols)
#     num_races = len(grouped_races)
#     print(f"Processing {num_races} races...")

#     for i, (race_info, race_df) in enumerate(grouped_races):
#         if (i + 1) % 10 == 0: # Print progress
#              print(f"  Processing race {i+1}/{num_races}...")

#         # Sort race by lap number just in case
#         race_df = race_df.sort_values('LapNumber')

#         # --- Determine the target vector for this race ---
#         # Get the final position for each driver in THIS race
#         # Group by driver, get the LAST recorded final position (should be consistent)
#         final_positions = race_df.groupby(driver_id_col)[target_col].last()

#         # Create a target score vector (e.g., higher score for better position)
#         target_scores = {}
#         for driver_id, position in final_positions.items():
#             # Simple scoring: Higher score is better. 
#             # I have put non finishers as 20
#             if position == 20:
#                 score = 0
#             else:
#                 score = 20 - position + 1
#             target_scores[int(driver_id)] = score # Ensure driver_id is int key

#         # Create the final target vector of size max_drivers
#         # Initialize with 0 (score for drivers not present or DNF)
#         race_target_vector = np.zeros(max_drivers)

#         for driver_id_encoded, score in target_scores.items():
#             # Place the score at the index matching the driver's global ID
#             if driver_id_encoded < max_drivers: # Safety check
#                 race_target_vector[driver_id_encoded] = score
#             # No 'else' needed, IDs beyond the known range are ignored

#         # --- Generate sequences for each driver in this race ---
#         drivers_in_race = race_df[driver_id_col].unique()
#         for driver_id in drivers_in_race:
#             driver_laps_df = race_df[race_df[driver_id_col] == driver_id].sort_values('LapNumber')
#             max_lap = driver_laps_df['LapNumber'].max()

#             # Generate sequence for each lap completed by the driver
#             for k in range(1, int(max_lap) + 1):
#                 # Get data for laps 1 to k for this driver
#                 current_sequence_df = driver_laps_df[driver_laps_df['LapNumber'] <= k]

#                 # Extract numerical features for the sequence
#                 numerical_sequence = current_sequence_df[numerical_cols].values
                
#                 all_numerical_sequences.append(numerical_sequence)

#                 # Extract each ID feature for the sequence
#                 for idx, id_col in enumerate(id_cols):
#                     id_sequence = current_sequence_df[id_col].values
                    
#                     all_id_sequences[idx].append(id_sequence)

#                 # Extract Active Flag for drivers
#                 active_flag_sequence = current_sequence_df[active_flag_col].values.astype(np.float32)
#                 all_active_flags.append(active_flag_sequence)

#                 # The target is the same for all sequences from this race
#                 all_targets.append(race_target_vector)

#     print(f"Finished generating {len(all_targets)} sequences.")
#     # Return numerical sequences, a list containing lists of sequences for each ID type, and targets
#     return all_numerical_sequences, all_id_sequences, all_active_flags, all_targets

# def prediction(pred, y_test):
#     drivers_mapping = mappings.drivers.copy()
#     driver_scores_prediction = {}
#     driver_scores_actual = {}

#     for name , id in drivers_mapping.items():
#         driver_scores_prediction[name] = round(list(pred)[0][id], 2)
#         driver_scores_actual[name] = int(list(y_test)[0][id])

#     print(driver_scores_prediction)
#     print(driver_scores_actual)
#     return driver_scores_prediction, driver_scores_actual

# def final_forecast_ranking(driver_scores_prediction, driver_scores_actual):
#     df_pred = pd.DataFrame(sorted(driver_scores_prediction.items(), key=lambda item: item[1], reverse=True), columns=['Driver', 'Score'])
#     df_actual = pd.DataFrame(sorted(driver_scores_actual.items(), key=lambda item: item[1], reverse=True), columns=['Driver', 'Score'])
#     dff = pd.concat([df_pred, df_actual], axis=1)
#     dff.columns= ['Driver_Pred', 'Score_Pred', 'Driver_Actual', 'Score_Actual']
#     return dff

# # --- Evaluate on Test Set (Optional but Recommended) ---
# # 1. Generate test sequences and pad them using the SAME scaler and MAX_SEQUENCE_LENGTH
# MAX_SEQUENCE_LENGTH = 78
# test_df = test_part_df.copy()
# test_df[numerical_feature_cols] = scaler.transform(test_df[numerical_feature_cols])

# print("\nPreparing Test Data...")
# X_test_num_seq, X_test_id_seq_lists, X_test_active_flags, y_test_list = generate_sequences(
#     test_df, numerical_feature_cols, [driver_id_col, team_id_col, compound_id_col, track_status_col, track_id_col, position_col, grid_position_col],
#     driver_active_mask_col,
#     target_col, race_id_cols, NUM_DRIVERS
# )

# X_test_num_padded = pad_sequences(X_test_num_seq, maxlen=MAX_SEQUENCE_LENGTH, padding='pre', dtype='float32', value=0.0)
# X_test_ids_padded = [
#     pad_sequences(id_list, maxlen=MAX_SEQUENCE_LENGTH, padding='pre', value=0)
#     for id_list in X_test_id_seq_lists
# ]

# X_test_active_padded = pad_sequences(X_test_active_flags, maxlen=MAX_SEQUENCE_LENGTH, padding='pre', dtype='float32', value=0.0)

# y_test = np.array(y_test_list)
# X_test_inputs = [X_test_num_padded] + X_test_ids_padded + [X_test_active_padded]
# # X_test_inputs_dict = { ... } # Dictionary version

# # 2. Load the best saved model
# best_model = keras.models.load_model("best_f1_model1.keras") # error here because of the name

# # 3. Evaluate
# test_loss, test_mae = best_model.evaluate(X_test_inputs, y_test) # error here because of the name
# print(f"\nTest Loss: {test_loss:.4f}")
# print(f"Test MAE: {test_mae:.4f}")

# pred = best_model.predict(X_test_inputs)
# pred, actual = prediction(pred, y_test)
# dff = final_forecast_ranking(pred, actual)
# dff

In [585]:
df = pd.read_csv('final_data_2025.csv')

In [586]:
df.isna().sum()

LapNumber               0
Driver                  0
LapTime                 0
Compound                0
TyreLife                0
Stint                   0
PitInTime               0
PitOutTime              0
FreshTyre               0
Team                  273
TrackStatus             0
Position                0
Sector1Time             0
Sector2Time             0
Sector3Time             0
SpeedI1                 0
SpeedI2                 0
SpeedFL                 0
SpeedST                 0
SessionTime             0
AirTemp                 0
Humidity                0
Pressure                0
Rainfall                0
TrackTemp               0
WindDirection           0
WindSpeed               0
RpmAvg                  0
RpmMin                  0
RpmMax                  0
SpeedAvg                0
SpeedMedian             0
SpeedMin                0
SpeedMax                0
ThrottleAvg             0
ThrottleMin             0
ThrottleMax             0
nGearAvg                0
nGearMin    

In [415]:
test_part_df = df.loc[(df['Year'] == 2025)&(test_part_df['Location'] == 'Suzuka')&(df['Driver']==31)&(test_part_df['LapNumber'] < 10)].copy()

In [592]:
scaler = joblib.load('scaler.pkl')


# --- Hyperparameters ---
NUM_DRIVERS = 21

# --- columns ---
numerical_feature_cols = ['LapTime', 'TyreLife', 'Stint','FreshTyre',  'Sector1Time','Sector2Time', 'Sector3Time', 'SpeedI1', 'SpeedI2', 'SpeedFL',
    'SpeedST', 'SessionTime', 'AirTemp', 'Humidity', 'Pressure', 'Rainfall','TrackTemp', 'WindDirection', 'WindSpeed', 'RpmAvg', 'RpmMin', 'RpmMax',
    'SpeedAvg', 'SpeedMedian', 'SpeedMin', 'SpeedMax', 'ThrottleAvg','ThrottleMin', 'ThrottleMax', 'nGearAvg', 'nGearMin', 'nGearMax',
    'BrakeCount', 'DrsCount', 'nGearMode',   
    # 'Age', 'Exprience','Achievements', 'AchievementsByTime',
    # 'LastYearDriverPoints', 'LastYearTeamPoints',
    # 'Position', 'GridPosition',
    ]
driver_id_col = 'Driver'
driver_active_mask_col = 'IsDriverActive'
team_id_col = 'Team'
compound_id_col = 'Compound'
track_status_col = 'TrackStatus'
track_id_col = 'Track'
position_col = 'Position'
grid_position_col = 'GridPosition'

# --- Define Race Identifier ---
# Use a combination of columns that uniquely identifies each race event
race_id_cols = ['Year', 'Location'] 
# --- Define Target Column ---
target_col = 'FinalPosition'

# --- Function to Generate Sequences and Targets ---
def generate_sequences(df, numerical_cols, id_cols, 
                    #    active_flag_col, 
                       target_col, 
                       race_group_cols, max_drivers):
    """
    Generates sequences lap-by-lap for each driver in each race.

    Args:
        df (pd.DataFrame): DataFrame containing lap data for multiple races.
        numerical_cols (list): List of numerical feature column names.
        id_cols (list): List of categorical ID column names (already encoded).
        target_col (str): Name of the column containing the final driver position.
        race_group_cols (list): Columns to group by to identify individual races.
        max_drivers (int): The fixed number of drivers for the output vector.

    Returns:
        tuple: (list_of_numerical_sequences, list_of_id_sequences_lists, list_of_target_vectors)
               - list_of_id_sequences_lists contains one list per ID type.
    """
    all_numerical_sequences = []
    # Create a list of lists, one for each ID type
    all_id_sequences = [[] for _ in id_cols]
    all_targets = []
    all_active_flags = []

    # Group by each unique race
    grouped_races = df.groupby(race_group_cols)
    num_races = len(grouped_races)
    print(f"Processing {num_races} races...")

    for i, (race_info, race_df) in enumerate(grouped_races):
        if (i + 1) % 10 == 0: # Print progress
             print(f"  Processing race {i+1}/{num_races}...")

        # Sort race by lap number just in case
        race_df = race_df.sort_values('LapNumber')

        # --- Generate sequences for each driver in this race ---
        drivers_in_race = race_df[driver_id_col].unique()
        for driver_id in drivers_in_race:
            
            # probability target vector for this driver
            race_target_vector = np.zeros(max_drivers)
            final_position = race_df.loc[race_df[driver_id_col] == driver_id, target_col].iloc[-1]

            print(final_position)
            race_target_vector[final_position] = 1

            driver_laps_df = race_df[race_df[driver_id_col] == driver_id].sort_values('LapNumber')
            max_lap = driver_laps_df['LapNumber'].max()

            # Generate sequence for each lap completed by the driver
            for k in range(1, int(max_lap) + 1):
                # Get data for laps 1 to k for this driver
                current_sequence_df = driver_laps_df[driver_laps_df['LapNumber'] <= k]

                # Extract numerical features for the sequence
                numerical_sequence = current_sequence_df[numerical_cols].values
                
                all_numerical_sequences.append(numerical_sequence)

                # Extract each ID feature for the sequence
                for idx, id_col in enumerate(id_cols):
                    id_sequence = current_sequence_df[id_col].values
                    
                    all_id_sequences[idx].append(id_sequence)

                # The target is the same for all sequences from this race
                all_targets.append(race_target_vector) 

    print(f"Finished generating {len(all_targets)} sequences.")
    # Return numerical sequences, a list containing lists of sequences for each ID type, and targets
    return all_numerical_sequences, all_id_sequences, all_targets

def prediction(pred, y_test):
    drivers_mapping = mappings.drivers.copy()
    driver_scores_prediction = {}
    driver_scores_actual = {}

    for name , id in drivers_mapping.items():
        driver_scores_prediction[name] = round(list(pred)[0][id], 2)
        driver_scores_actual[name] = int(list(y_test)[0][id])

    print(driver_scores_prediction)
    print(driver_scores_actual)
    return driver_scores_prediction, driver_scores_actual

def final_forecast_ranking(driver_scores_prediction, driver_scores_actual):
    df_pred = pd.DataFrame(sorted(driver_scores_prediction.items(), key=lambda item: item[1], reverse=False), columns=['Driver', 'Rank'])
    df_actual = pd.DataFrame(sorted(driver_scores_actual.items(), key=lambda item: item[1], reverse=False), columns=['Driver', 'Rank'])
    dff = pd.concat([df_pred, df_actual], axis=1)
    dff.columns= ['Driver_Pred', 'FinalPosition_Pred', 'Driver_Actual', 'FinalPosition_Actual']
    return dff


def test_model(df):
    # --- Evaluate on Test Set (Optional but Recommended) ---
    # 1. Generate test sequences and pad them using the SAME scaler and MAX_SEQUENCE_LENGTH
    MAX_SEQUENCE_LENGTH = 78
    test_df = df.copy()
    test_df[numerical_feature_cols] = scaler.transform(test_df[numerical_feature_cols])

    print("\nPreparing Test Data...")
    X_test_num_seq, X_test_id_seq_lists, y_test_list = generate_sequences(
        test_df, numerical_feature_cols, [
                                        # driver_id_col, 
                                        # team_id_col, 
                                        # compound_id_col, 
                                        # track_status_col, 
                                        track_id_col, 
                                        # position_col, 
                                        grid_position_col
                                        ],
        # driver_active_mask_col,
        target_col, race_id_cols, NUM_DRIVERS
    )

    X_test_num_padded = pad_sequences(X_test_num_seq, maxlen=MAX_SEQUENCE_LENGTH, padding='pre', dtype='float32', value=0.0)
    X_test_ids_padded = [
        pad_sequences(id_list, maxlen=MAX_SEQUENCE_LENGTH, padding='pre', value=0)
        for id_list in X_test_id_seq_lists
    ]

    # X_test_active_padded = pad_sequences(X_test_active_flags, maxlen=MAX_SEQUENCE_LENGTH, padding='pre', dtype='float32', value=0.0)

    y_test = np.array(y_test_list)
    X_test_inputs = [X_test_num_padded] + X_test_ids_padded # + [X_test_active_padded]
    # X_test_inputs_dict = { ... } # Dictionary version

    # 2. Load the best saved model
    best_model = keras.models.load_model("best_f1_model1.keras") # error here because of the name

    # 3. Evaluate
    test_loss, test_accuracy, test_top_3_accuracy = best_model.evaluate(X_test_inputs, y_test) # error here because of the name
    print(f"\nTest Loss: {test_loss:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f}")
    print(f"Test Top 3 Accuracy: {test_top_3_accuracy:.4f}")

    pred = best_model.predict(X_test_inputs)
    # pred, actual = prediction(pred, y_test)
    # dff = final_forecast_ranking(pred, actual)
    # dff
    # print(pred)
    return pred, y_test

In [591]:
def get_value_sorted_by_position(arr):
    # arr = np.array(arr)
    sorted_indices = np.argsort(arr)[::-1]  # Indices that sort the array in descending order
    return {int(idx): round(float(arr[idx]), 3) for idx in sorted_indices}

def get_value_sorted_by_position_highest_probability(arr):
    sorted_indices = np.argsort(arr)[::-1]
    return int(sorted_indices[0])

In [596]:
df = df.loc[df['LapTime'] < 10]

In [598]:
drivers=df['Driver'].unique()
driver_scores_prediction = {}
driver_scores_actual = {}
drivers_mapping = mappings.drivers.copy()

## parameters
NUMBER_OF_LAPS_TO_GO_INTO_MODEL = 49
RACE_NAME='Melbourne'

for driver in drivers:
    test_part_df = df.loc[(df['Year'] == 2025)&(df['Location'] == RACE_NAME)&(df['Driver']==driver)&(df['LapNumber'] <= NUMBER_OF_LAPS_TO_GO_INTO_MODEL)].copy()
    if not test_part_df.empty:
        pred, y_test = test_model(test_part_df)
        for name , id in drivers_mapping.items():
            if id == driver:
                result = get_value_sorted_by_position_highest_probability(pred[-1])
                driver_scores_prediction[name] = result
                result = get_value_sorted_by_position_highest_probability(y_test[-1])
                driver_scores_actual[name] = result

dff = final_forecast_ranking(driver_scores_prediction, driver_scores_actual)
dff


Preparing Test Data...
Processing 1 races...
2
Finished generating 45 sequences.
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 118ms/step - accuracy: 0.8991 - loss: 1.9019 - top_3_accuracy: 0.9243

Test Loss: 1.8482
Test Accuracy: 0.9111
Test Top 3 Accuracy: 0.9333
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 644ms/step

Preparing Test Data...
Processing 1 races...
11
Finished generating 45 sequences.
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 107ms/step - accuracy: 0.0000e+00 - loss: 2.6134 - top_3_accuracy: 0.0653

Test Loss: 2.5891
Test Accuracy: 0.0000
Test Top 3 Accuracy: 0.0667
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 662ms/step

Preparing Test Data...
Processing 1 races...
4
Finished generating 46 sequences.
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 105ms/step - accuracy: 0.0000e+00 - loss: 5.3353 - top_3_accuracy: 0.0000e+00

Test Loss: 5.3367
Test Accuracy: 0.0000
Test Top 3 Accuracy: 0.0000
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 648ms/step

Preparing Test Data...
Processing 1 races...
20
Finished generating 32 sequences.
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - ac

,Driver_Pred,FinalPosition_Pred,Driver_Actual,FinalPosition_Actual
0,NOR,1,NOR,1
1,VER,2,VER,2
2,RUS,2,RUS,3
3,PIA,2,ANT,4
4,TSU,6,ALB,5
5,ALB,7,STR,6
6,GAS,10,HUL,7
7,ANT,10,LEC,8
8,ALO,10,PIA,9
9,LEC,10,HAM,10


This prediction is not perfect but remember: 

I don't even feed the live position of the drivers to the model. 
I only feed the car data , sector and laptimes. 
If I include the current position of the drivers, the prediction will be much more accurate.

In [583]:
pred.shape

(50, 21)